In [1]:
from pathlib import Path
from shapely import Point
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

top_level = Path.cwd()

while (top_level / "pyproject.toml").exists() is False:
    top_level = top_level.parent

parl_data_loc = (
    top_level
    / "data"
    / "packages"
    / "parliament_con_2025"
    / "parl_constituencies_2025.parquet"
)

In [2]:
codepoint_dir = top_level / "data" / "raw" / "external" / "codepo"
csv_dir = codepoint_dir / "csv"
parquet_dir = codepoint_dir / "parquet"

header_file = codepoint_dir / "header.csv"

parl_data = gpd.read_parquet(parl_data_loc).to_crs(epsg=4326)
output_loc = (
    top_level
    / "data"
    / "packages"
    / "uk_parliament_2025_postcode_lookup"
    / "postcodes_with_con.parquet"
)
all_data = []


# read in first line of header file as comma separated list
with open(header_file, "r") as f:
    header = f.readline().strip().split(",")

# for each file in csv_dir - read in as dataframe using header and write out as parquet with a new geometry column


for csv_file in tqdm(list(csv_dir.glob("*.csv"))):
    df = pd.read_csv(csv_file, names=header)
    # from Eastings,Northings, we want to create a geopandas dataframe with a geometry column of point data
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["Eastings"], df["Northings"]),
        crs="EPSG:27700",
    )
    # gdf.to_parquet(parquet_dir / csv_file.with_suffix(".parquet").name)
    gdf = gdf.to_crs(epsg=4326)

    joined = gpd.sjoin(gdf, parl_data, how="left", predicate="within")[
        ["Postcode", "short_code"]
    ]
    joined = joined.rename(columns={"Postcode": "postcode"})

    all_data.append(joined)

df = pd.concat(all_data)

# sort by postcode
df = df.sort_values("postcode")

df.to_parquet(output_loc)

  0%|                                                                                                                                          | 0/120 [00:00<?, ?it/s]

  1%|█                                                                                                                                 | 1/120 [00:00<00:16,  7.30it/s]

  2%|███▎                                                                                                                              | 3/120 [00:00<00:10, 10.68it/s]

  4%|█████▍                                                                                                                            | 5/120 [00:00<00:09, 12.40it/s]

  6%|███████▌                                                                                                                          | 7/120 [00:00<00:09, 11.81it/s]

...

 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌      | 114/120 [00:09<00:00, 11.36it/s]

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋    | 116/120 [00:10<00:00, 12.62it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 118/120 [00:10<00:00, 12.29it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:10<00:00, 12.03it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:10<00:00, 11.56it/s]